In [1]:
# Question 01
import math

X = "X"
O = "O"
EMPTY = None

class TicTacToe:
    def __init__(self):
        self.board = [[O, EMPTY, X],
                      [X, EMPTY, EMPTY],
                      [X, O, O]]
        self.current_player = X
        self.game_over = False

    def play_game(self):
        while not self.game_over:
            self.print_board()

            winner = self.get_winner()
            if winner is not None:
                print(f"Player {winner} wins!")
                self.game_over = True
                break
            elif all(all(row) for row in self.board):
                print("Tie game!")
                self.game_over = True
                break

            if self.current_player == X:
                best_score = -math.inf
                best_move = None
                for i in range(3):
                    for j in range(3):
                        if self.board[i][j] == EMPTY:
                            self.board[i][j] = X
                            score = self.minimax(0, False)
                            self.board[i][j] = EMPTY
                            if score > best_score:
                                best_score = score
                                best_move = (i, j)
                self.board[best_move[0]][best_move[1]] = X
            else:
                best_score = math.inf
                best_move = None
                for i in range(3):
                    for j in range(3):
                        if self.board[i][j] == EMPTY:
                            self.board[i][j] = O
                            score = self.minimax(0, True)
                            self.board[i][j] = EMPTY
                            if score < best_score:
                                best_score = score
                                best_move = (i, j)
                self.board[best_move[0]][best_move[1]] = O

            self.current_player = O if self.current_player == X else X

    def minimax(self, depth, is_maximizing):
        winner = self.get_winner()
        if winner is not None:
            return self.score(depth)

        if is_maximizing:
            best_score = -math.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == EMPTY:
                        self.board[i][j] = X
                        score = self.minimax(depth+1, False)
                        self.board[i][j] = EMPTY
                        best_score = max(score, best_score)
            return best_score

        else:
            best_score = math.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == EMPTY:
                        self.board[i][j] = O
                        score = self.minimax(depth+1, True)
                        self.board[i][j] = EMPTY
                        best_score = min(score, best_score)
            return best_score

    def get_winner(self):
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != EMPTY:
                return self.board[i][0]

        for j in range(3):
            if self.board[0][j] == self.board[1][j] == self.board[2][j] != EMPTY:
                return self.board[0][j]

        if self.board[0][0] == self.board[1][1] == self.board[2][2] != EMPTY:
            return self.board[0][0]
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != EMPTY:
            return self.board[0][2]

        return None

    def score(self, depth):
        winner = self.get_winner()
        if winner == X:
            return 10 - depth
        elif winner == O:
            return depth - 10
        else:
            return 0

    def print_board(self):
        print("-------------")
        for row in self.board:
            print("|", end="")
            for cell in row:
                if cell is None:
                    print("   ", end="|")
                else:
                    print(f" {cell} ", end="|")
            print("\n-------------")

game = TicTacToe()
game.play_game()


-------------
| O |   | X |
-------------
| X |   |   |
-------------
| X | O | O |
-------------
-------------
| O |   | X |
-------------
| X | X |   |
-------------
| X | O | O |
-------------
Player X wins!


In [2]:
# Question 02
class Minimax:
    def __init__(self, values):
        self.values = values
        self.max = 1000
        self.min = -1000

    def minimax(self, depth, nodeIndex, maximizingPlayer, alpha, beta):
        if depth == 3:
            return self.values[nodeIndex]

        if maximizingPlayer:
            best = self.min
            for i in range(0, 2):
                val = self.minimax(depth + 1, nodeIndex * 2 + i, False, alpha, beta)
                best = max(best, val)
                alpha = max(alpha, best)
                if beta <= alpha:
                    break

            return best

        else: 
            best = self.max
            for i in range(0, 2):
                val = self.minimax(depth + 1, nodeIndex * 2 + i, True, alpha, beta)
                best = min(best, val)
                beta = min(beta, best)

                if beta <= alpha:
                    break

            return best

    def get_optimal_value(self):
        return self.minimax(0, 0, True, self.min, self.max)
        
values = [2, 4, 6, 8, 1, 2, 10, 12]
minimax = Minimax(values)
print("The optimal value is:", minimax.get_optimal_value())


The optimal value is: 4


In [4]:
# Question 03
class CSP:
    def __init__(self, variables, domains, constraints):
        self.variables = variables
        self.domains = domains
        self.constraints = constraints

    def solve(self):
        assignments = {}
        return self.backtrack(assignments)

    def backtrack(self, assignments):
        if len(assignments) == len(self.variables):
            return assignments

        unassigned = [v for v in self.variables if v not in assignments]
        var = self.select_unassigned_variable(unassigned, assignments)
        for value in self.order_domain_values(var, assignments):
            if self.is_consistent(var, value, assignments):
                assignments[var] = value
                result = self.backtrack(assignments)
                if result is not None:
                    return result
                del assignments[var]

        return None

    def select_unassigned_variable(self, variables, assignments):
        return variables[0]

    def order_domain_values(self, var, assignments):
        return self.domains[var]

    def is_consistent(self, var, value, assignments):
        assignments[var] = value
        for constraint in self.constraints:
            if not constraint(assignments):
                del assignments[var]
                return False
        del assignments[var]
        return True

def n_queens(n):
    variables = list(range(n))
    domains = {i: list(range(n)) for i in variables}
    constraints = [lambda a: len(set(a.values())) == n]
    for i, j in permutations(variables, 2):
        constraints.append(lambda a, i=i, j=j: abs(a[i] - a[j]) != abs(i - j))

    csp = CSP(variables, domains, constraints)
    result = csp.solve()
    if result is None:
        print("No solution found.")
    else:
        print("Solution:")
        for i in range(n):
            print(''.join(['Q' if result[i] == j else '.' for j in range(n)]))

n_queens(4)

NameError: name 'permutations' is not defined

In [5]:
# Question 04
def solve_cryptarithmetic():
    letters = {"b": 7, "a": 4, "s": 8, "e": 3, "l": 5, "g": 1, "m": 9}
    
    base = letters["b"]*1000 + letters["a"]*100 + letters["s"]*10 + letters["e"]
    ball = letters["b"]*1000 + letters["a"]*100 + letters["l"]*10 + letters["l"]
    games = letters["g"]*10000 + letters["a"]*1000 + letters["m"]*100 + letters["e"]*10 + letters["s"]
    
    result = base + ball
    if result == games:
        print("The solution is correct!")
        print("base =", base)
        print("ball =", ball)
        print("games =", games)
    else:
        print("The solution is incorrect.")

solve_cryptarithmetic()


The solution is correct!
base = 7483
ball = 7455
games = 14938
